In [1]:
# Downloaded libraries
import torch
from torch import nn
from torch_geometric.loader import DataLoader


# Local files
from dataset_graphs import NNDataset
from models import Trainer_GCN

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Constants
TRAINING_SPLIT = 0.8

In [3]:
# Hyperparameters
num_epoch = 100
batch_size = 16

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
nndataset = NNDataset(root="../")

size = len(nndataset)
train_num = int(size * TRAINING_SPLIT)
test_num = size - train_num

print(
    f"Dataset loaded, {train_num} training samples and {test_num} testing samples")

Processing...


Graph is Undirected
Reading from ..\raw\dataset_0/, 200 models loaded
Reading from ..\raw\dataset_1/, 200 models loaded
Reading from ..\raw\dataset_2/, 200 models loaded
Reading from ..\raw\dataset_3/, 200 models loaded
Reading from ..\raw\dataset_4/, 200 models loaded
Reading from ..\raw\dataset_5/, 200 models loaded
Dataset loaded, 960 training samples and 240 testing samples


Done!


In [16]:
data = nndataset[0]
print(data)
print(data.design)

Data(design=[6], edge_index=[2, 12864], x=[245, 503], y_node=[245, 1], y_edge=[6432, 1], input_mask=[245, 1], num_nodes=245)
[3, 32, 96, 16, 96, 2]


In [26]:
data.x[:,502]

tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0067e-01,  1.9384e-02,
         1.7750e-01, -4.6171e-01,  1.8293e-01, -2.4638e-01,  2.0000e-01,
         2.7218e-01,  3.0054e-01, -4.3329e-02,  2.6026e-01, -2.1197e-01,
         5.1550e-01,  2.8103e-01,  8.2824e-02,  2.6926e-01, -5.4662e-01,
         1.5758e-02, -5.5833e-01,  4.3861e-01,  1.8996e-01, -1.2367e-02,
         5.2220e-01, -2.1556e-01,  3.3489e-01,  7.0793e-03,  3.8506e-01,
        -4.3606e-01, -1.7828e-01, -2.0559e-01, -2.0938e-01, -2.3085e-01,
        -4.6774e-02,  1.5593e-01,  1.0852e-01, -1.0469e-01,  1.0978e-02,
        -2.9960e-02,  8.2496e-02, -1.3926e-02,  1.6312e-01, -1.5006e-01,
         4.0887e-02, -5.9743e-02, -7.2410e-03, -9.6327e-02,  1.2866e-01,
        -1.2609e-01, -1.5544e-01, -1.4454e-01, -1.5013e-01, -9.5906e-02,
        -1.2031e-01,  1.5928e-01, -8.3122e-02,  8.7837e-02, -7.7914e-02,
         1.4335e-01, -9.6918e-02,  1.4331e-01, -7.9432e-02, -1.5437e-02,
         1.1228e-01,  2.4273e-02, -9.8379e-02,  6.9

In [8]:
train_loader = DataLoader(
    dataset=nndataset[:train_num], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=nndataset[train_num:], batch_size=test_num, shuffle=True)

In [9]:
model = Trainer_GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.MSELoss()

In [10]:
# Model Training
model.train()
for epoch in range(num_epoch):
    print(f"Epoch {epoch + 1} / {num_epoch}:")
    for i, data in enumerate(train_loader):
        data.to(device)

        # forward propagation
        out_w, out_b = model(data)
        loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print status every n batches
        if i % 10 == 0:
            loss, current = loss.item(), i * batch_size
            print(
                f"Training Loss: {loss:>7f}  [{current:>5d}/{train_num:>5d}]")

Epoch 1 / 100:


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2063x503 and 504x256)

In [ ]:
# Model Evaluation
model.eval()
with torch.no_grad():
    data = iter(test_loader).next().to(device)

    # forward propagation
    out_w, out_b = model(data)
    loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

    loss = loss.item()
    print(f"Validation Loss: {loss:>7f}")


In [ ]:
data = nndataset[0]
data = data.to(device)

In [ ]:
out_w, out_b = model(data)

In [ ]:
torch.save(model.state_dict(), "../model/model")
print("Model saved")